 loading the my one file



In [ ]:
# %%
import pickle

# Load the .pkl file, specifying the encoding
with open('D:\WESAD\S3\S3.pkl', 'rb') as file:
    data_dict = pickle.load(file, encoding='latin1')  # Try 'latin1' or 'utf-8'

# Display the data
print(data_dict)


 just for calculating reading and combining the data, how it works

In [ ]:
# %%
import numpy as np
ecg_signal = data_dict['signal']['chest']['ECG'].flatten()  # Flatten to make it 1D array
label = data_dict['label']

# Combine ECG signal and label into a structured array
combined_data = np.zeros(len(ecg_signal), dtype=[('ECG', np.float64), ('label', np.int32)])
combined_data['ECG'] = ecg_signal
combined_data['label'] = label

# Print combined data
print("Combined ECG Signal and Label Data:")
print(combined_data)
print("Shape of combined data:", combined_data.shape)
import numpy as np


# Combine ECG signal and label into a 2D structured array
combined_data = np.empty((len(ecg_signal), ), dtype=[('ECG', np.float64), ('label', np.int32)])
combined_data['ECG'] = ecg_signal
combined_data['label'] = label

# Extract the rows where the label is 2
label_2_data = combined_data[combined_data['label'] == 2]
label_3_data = combined_data[combined_data['label'] == 3]
label_1_data = combined_data[combined_data['label'] == 1]

# Print the filtered data
print("Data corresponding to label 1:")
print(label_1_data)
size1=len(label_1_data)
print("size with label1",size1)


'''print("Data corresponding to label 2:")
print(label_2_data)'''
size2=len(label_2_data)
print("size with label2",size2)

#print("Data corresponding to label 3:")
size3=len(label_3_data)
print("size with label3",size3)



 calculating the epochs into user input time segments

In [ ]:
# %%
# Define the parameters

sampling_frequency = 700  # in Hz

# Function to calculate the number of segments
def calculate_segments(sample_size, segment_time):
    segment_samples = segment_time * sampling_frequency
    number_of_segments = sample_size / segment_samples
    return number_of_segments

# Get user input for the segment time
segment_time = float(input("Enter the segment time in seconds: "))

# Get user input for the three sample sizes
sample_size1 = size1
sample_size2 = size2
sample_size3 = size3

# Calculate the number of segments for each sample size
number_of_segments1 = calculate_segments(sample_size1, segment_time)
number_of_segments2 = calculate_segments(sample_size2, segment_time)
number_of_segments3 = calculate_segments(sample_size3, segment_time)

# Print the results
print(f"duration of each segment: {size1 / 700:.2f} seconds")
print(f"Number of segments for sample size {sample_size1}: {number_of_segments1}")

print(f"duration of each segment: {size2 / 700:.2f} seconds")
print(f"Number of segments for sample size {sample_size2}: {number_of_segments2}")

print(f"duration of each segment: {size3 / 700:.2f} seconds")
print(f"Number of segments for sample size {sample_size3}: {number_of_segments3}")




 collecting baseline,stress and amuse data from all files

In [ ]:
# %%
import os
import pickle
import numpy as np

# Define the directory path
directory_path = 'D:\\WESAD\\wesad4'

# Function to load data from a pickle file
def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

# Get a list of all pickle files in the directory
all_files = [f for f in os.listdir(directory_path) if f.endswith('.pkl')]



# Initialize a list to hold the loaded data
loaded_data1 = []
loaded_data2 = []
loaded_data3 = []

# Iterate through the selected pickle files and load them
for file_name in all_files:
    file_path = os.path.join(directory_path, file_name)
    data = load_pickle_file(file_path)
    
    ecg_signal = data['signal']['chest']['ECG'].flatten()  # Flatten to make it 1D array
    label = data['label']

# Combine ECG signal and label into a structured array
    combined_data = np.zeros(len(ecg_signal),dtype=[('ECG', np.float64), ('label', np.int32)])
    combined_data['ECG'] = ecg_signal
    combined_data['label'] = label

# Print combined data

    '''print("Combined ECG Signal and Label Data:")
    print(combined_data)
    print("Shape of combined data:", combined_data.shape)
    '''
# Extract the rows where the label is 2
    label_1_data = combined_data[combined_data['label'] == 1]
    label_2_data = combined_data[combined_data['label'] == 2]
    label_3_data = combined_data[combined_data['label'] == 3]
    '''print(label_1_data)
    print(label_2_data)
    print(label_3_data)'''
    loaded_data1.append(label_1_data)
    loaded_data2.append(label_2_data)
    loaded_data3.append(label_3_data)
'''print(loaded_data1)
print(loaded_data2)
print(loaded_data3)'''

# Extract ECG values
ecg_values1 = [arr['ECG'] for arr in loaded_data1]
ecg_values2 = [arr['ECG'] for arr in loaded_data2]
ecg_values3 = [arr['ECG'] for arr in loaded_data3]

# ecg_values now contains a list of ECG values from each array

print(len(ecg_values1))
print(len(ecg_values2))
print(len(ecg_values3))

baseline_arrayy = np.concatenate(ecg_values1)
stress_arrayy = np.concatenate(ecg_values2)
amuse_arrayy = np.concatenate(ecg_values3)

print(baseline_arrayy)
print(stress_arrayy)
print(amuse_arrayy)
print("length of baseline samples ",len(baseline_arrayy))
print("length of stress sample ",len(stress_arrayy))
print("length of amuse sample",len(amuse_arrayy))


 Preprocessing the data

In [ ]:
# %%
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
def preprocess_ecg(ecg_signal, fs):
    # Bandpass filter to remove noise
    lowcut = 0.5  # 0.5 Hz
    highcut = 45.0  # 45 Hz
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(1, [low, high], btype='band')
    filtered_ecg = signal.filtfilt(b, a, ecg_signal)
    
    normalized_ecg = (filtered_ecg - np.mean(filtered_ecg)) / np.std(filtered_ecg)
    return normalized_ecg



In [ ]:
# %%
fs = 700 # Sampling frequency (Hz)
ecg_signal = baseline_arrayy # Load your ECG data
baseline_array = preprocess_ecg(ecg_signal, fs)

time = np.arange(0, len(ecg_signal)) / fs
plt.figure(figsize=(12,10)) 




# Define the zoom range in seconds
zoom_start = 0
zoom_end = 10  # Zoom into the first 10 seconds

# Plot original ECG signal
plt.subplot(2, 1, 1)
plt.plot(time, ecg_signal, label='Original ECG Signal')
plt.xlim(zoom_start, zoom_end)
plt.title('Original Baseline ECG Signal (Zoomed In)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

# Plot normalized ECG signal
plt.subplot(2, 1, 2)
plt.plot(time,baseline_array, label='Normalized ECG Signal', color='orange')
plt.xlim(zoom_start, zoom_end)
plt.title('Normalized Baseline ECG Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (normalized)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# %%
import numpy as np
import scipy.signal as signal
def preprocess_ecg(ecg_signal, fs):
    # Bandpass filter to remove noise
    lowcut = 0.5  # 0.5 Hz
    highcut = 45.0  # 45 Hz
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(1, [low, high], btype='band')
    filtered_ecg = signal.filtfilt(b, a, ecg_signal)
    normalized_ecg = (filtered_ecg - np.mean(filtered_ecg)) / np.std(filtered_ecg)
    return normalized_ecg

fs = 700 # Sampling frequency (Hz)
ecg_signal = stress_arrayy # Load your ECG data
stress_array = preprocess_ecg(ecg_signal, fs)


In [ ]:
# %%
fs = 700 # Sampling frequency (Hz)
ecg_signal = stress_arrayy # Load your ECG data
stress_array = preprocess_ecg(ecg_signal, fs)

time = np.arange(0, len(ecg_signal)) / fs

plt.figure(figsize=(12,10)) 




# Define the zoom range in seconds
zoom_start = 0
zoom_end = 10  # Zoom into the first 10 seconds

# Plot original ECG signal
plt.subplot(2, 1, 1)
plt.plot(time, ecg_signal, label='Original ECG Signal')
plt.xlim(zoom_start, zoom_end)
plt.title('Original Stress ECG Signal (Zoomed In)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

# Plot normalized ECG signal
plt.subplot(2, 1, 2)
plt.plot(time,stress_array, label='Normalized ECG Signal', color='orange')
plt.xlim(zoom_start, zoom_end)
plt.title('Normalized  Stress ECG Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (normalized)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# %%
import numpy as np
import scipy.signal as signal
def preprocess_ecg(ecg_signal, fs):
    # Bandpass filter to remove noise
    lowcut = 0.5  # 0.5 Hz
    highcut = 45.0  # 45 Hz
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(1, [low, high], btype='band')
    filtered_ecg = signal.filtfilt(b, a, ecg_signal)
    normalized_ecg = (filtered_ecg - np.mean(filtered_ecg)) / np.std(filtered_ecg)
    return normalized_ecg

fs = 700 # Sampling frequency (Hz)
ecg_signal = amuse_arrayy # Load your ECG data
amuse_array = preprocess_ecg(ecg_signal, fs)


In [ ]:
# %%
fs = 700 # Sampling frequency (Hz)
ecg_signal = amuse_arrayy
amuse_array = preprocess_ecg(ecg_signal, fs)

time = np.arange(0, len(ecg_signal)) / fs
plt.figure(figsize=(12,10)) 


zoom_start = 0
zoom_end = 10  

# Plot original ECG signal
plt.subplot(2, 1, 1)
plt.plot(time, ecg_signal, label='Original ECG Signal')
plt.xlim(zoom_start, zoom_end)
plt.title('Original  Amusement ECG Signal (Zoomed In)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

# Plot normalized ECG signal
plt.subplot(2, 1, 2)
plt.plot(time,amuse_array, label='Normalized ECG Signal', color='orange')
plt.xlim(zoom_start, zoom_end)
plt.title('Normalized Amusement ECG Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (normalized)')
plt.legend()

plt.tight_layout()
plt.show()


 segmentation and feature extraction

In [ ]:
# %%
pip install numpy scipy biosppy matplotlib



In [ ]:
# %%
import numpy as np
import scipy.stats as stats
from scipy.signal import welch

# Example ECG signal (replace this with your actual signal)
 # Example data, replace with actual ECG signal

# Input for window size and sampling rate
window_size_seconds = int(input("Enter a window or segment size in seconds: ")) 
sampling_rate = int(input("Enter sampling frequency in Hz: "))

# Convert window size to number of samples
window_size_samples = window_size_seconds * sampling_rate

# Segment the signal into non-overlapping windows
segments = []
for start in range(0, len(baseline_array), window_size_samples):
    end = start + window_size_samples
    if end <= len(baseline_array):
        segment = baseline_array[start:end]
        segments.append(segment)

def extract_features(segment, fs):
    """
    Extract features from a segment of time-series data.
    
    Parameters:
        segment (numpy array): The input time-series segment.
        fs (int): Sampling frequency of the data in Hz.

    Returns:
        list: A list containing the extracted features.
    """
    features = []
    
    # Time-domain features
    features.append(np.mean(segment))
    features.append(np.std(segment))
    features.append(np.min(segment))
    features.append(np.max(segment))
    
    nn_intervals = np.diff(segment)
    nn50 = sum(np.abs(nn_intervals) > 50)
    rmssd = np.sqrt(np.mean(nn_intervals ** 2))
    features.append(nn50)
    features.append(rmssd)
    
    triangular_index = len(segment) / np.max(np.histogram(segment, bins='auto')[0])
    features.append(triangular_index)
    
    features.append(np.sqrt(np.mean(segment**2)))  # RMS
    features.append(stats.skew(segment))
    features.append(stats.kurtosis(segment))
    
    # Frequency-domain features using Welch’s method
    f, Pxx = welch(segment, fs=fs, nperseg=len(segment)//8)
    power_low = np.trapz(Pxx[(f >= 0.5) & (f < 4)])
    power_mid = np.trapz(Pxx[(f >= 4) & (f < 8)])
    power_high = np.trapz(Pxx[(f >= 8) & (f < 13)])
    
    features.append(power_low)
    features.append(power_mid)
    features.append(power_high)
    
    return features

feature_names = [
    "Mean", "Standard Deviation", "Minimum", "Maximum", "NN50",
    "RMSSD", "Triangular Index", "RMS",
    "Skewness", "Kurtosis", "power_low", 
    "power_mid", "power_high"
]

# Extract features from each segment
extracted_features = []
for segment in segments:
    features = extract_features(segment, sampling_rate)
    extracted_features.append(features)

# Convert the list of features to a NumPy array
baseline_extracted_features = np.array(extracted_features)

# Print the extracted features
print("Feature Names:")
print(feature_names)
print("\nExtracted Features Shape:", baseline_extracted_features.shape)
print("Extracted Features for baseline array:")
print(baseline_extracted_features)



In [ ]:
# %%
import numpy as np
import scipy.stats as stats
from scipy.signal import welch

# Example ECG signal (replace this with your actual signal)
 # Example data, replace with actual ECG signal

# Input for window size and sampling rate
window_size_seconds = int(input("Enter a window or segment size in seconds: ")) 
sampling_rate = int(input("Enter sampling frequency in Hz: "))

# Convert window size to number of samples
window_size_samples = window_size_seconds * sampling_rate

# Segment the signal into non-overlapping windows
segments = []
for start in range(0, len(stress_array), window_size_samples):
    end = start + window_size_samples
    if end <= len(stress_array):
        segment = stress_array[start:end]
        segments.append(segment)

def extract_features(segment, fs):
    """
    Extract features from a segment of time-series data.
    
    Parameters:
        segment (numpy array): The input time-series segment.
        fs (int): Sampling frequency of the data in Hz.

    Returns:
        list: A list containing the extracted features.
    """
    features = []
    
    # Time-domain features
    features.append(np.mean(segment))
    features.append(np.std(segment))
    features.append(np.min(segment))
    features.append(np.max(segment))
    
    nn_intervals = np.diff(segment)
    nn50 = sum(np.abs(nn_intervals) > 50)
    rmssd = np.sqrt(np.mean(nn_intervals ** 2))
    features.append(nn50)
    features.append(rmssd)
    
    triangular_index = len(segment) / np.max(np.histogram(segment, bins='auto')[0])
    features.append(triangular_index)
    
    features.append(np.sqrt(np.mean(segment**2)))  # RMS
    features.append(stats.skew(segment))
    features.append(stats.kurtosis(segment))
    
    # Frequency-domain features using Welch’s method
    f, Pxx = welch(segment, fs=fs, nperseg=len(segment)//8)
    power_low = np.trapz(Pxx[(f >= 0.5) & (f < 4)])
    power_mid = np.trapz(Pxx[(f >= 4) & (f < 8)])
    power_high = np.trapz(Pxx[(f >= 8) & (f < 13)])
    
    features.append(power_low)
    features.append(power_mid)
    features.append(power_high)
    
    return features

feature_names = [
    "Mean", "Standard Deviation", "Minimum", "Maximum", "NN50",
    "RMSSD", "Triangular Index", "RMS",
    "Skewness", "Kurtosis", "power_low", 
    "power_mid", "power_high"
]

# Extract features from each segment
extracted_features = []
for segment in segments:
    features = extract_features(segment, sampling_rate)
    extracted_features.append(features)

# Convert the list of features to a NumPy array
stress_extracted_features = np.array(extracted_features)

# Print the extracted features
print("Feature Names:")
print(feature_names)
print("\nExtracted Features Shape:", stress_extracted_features.shape)
print("Extracted Features for stress array:")
print(stress_extracted_features)



In [ ]:
# %%
import numpy as np
import scipy.stats as stats
from scipy.signal import welch

# Example ECG signal (replace this with your actual signal)
 # Example data, replace with actual ECG signal

# Input for window size and sampling rate
window_size_seconds = int(input("Enter a window or segment size in seconds: ")) 
sampling_rate = int(input("Enter sampling frequency in Hz: "))

# Convert w10
# indow size to number of samples
window_size_samples = window_size_seconds * sampling_rate

# Segment the signal into non-overlapping windows
segments = []
for start in range(0, len(amuse_array), window_size_samples):
    end = start + window_size_samples
    if end <= len(amuse_array):
        segment = amuse_array[start:end]
        segments.append(segment)

def extract_features(segment, fs):
    """
    Extract features from a segment of time-series data.
    
    Parameters:
        segment (numpy array): The input time-series segment.
        fs (int): Sampling frequency of the data in Hz.

    Returns:
        list: A list containing the extracted features.
    """
    features = []
    
    # Time-domain features
    features.append(np.mean(segment))
    features.append(np.std(segment))
    features.append(np.min(segment))
    features.append(np.max(segment))
    
    nn_intervals = np.diff(segment)
    nn50 = sum(np.abs(nn_intervals) > 50)
    rmssd = np.sqrt(np.mean(nn_intervals ** 2))
    features.append(nn50)
    features.append(rmssd)
    
    triangular_index = len(segment) / np.max(np.histogram(segment, bins='auto')[0])
    features.append(triangular_index)
    
    features.append(np.sqrt(np.mean(segment**2)))  # RMS
    features.append(stats.skew(segment))
    features.append(stats.kurtosis(segment))
    
    # Frequency-domain features using Welch’s method
    f, Pxx = welch(segment, fs=fs, nperseg=len(segment)//8)
    power_low = np.trapz(Pxx[(f >= 0.5) & (f < 4)])
    power_mid = np.trapz(Pxx[(f >= 4) & (f < 8)])
    power_high = np.trapz(Pxx[(f >= 8) & (f < 13)])
    
    features.append(power_low)
    features.append(power_mid)
    features.append(power_high)
    
    return features

feature_names = [
    "Mean", "Standard Deviation", "Minimum", "Maximum", "NN50",
    "RMSSD", "Triangular Index", "RMS",
    "Skewness", "Kurtosis", "power_low", 
    "power_mid", "power_high"
]

# Extract features from each segment
extracted_features = []
for segment in segments:
    features = extract_features(segment, sampling_rate)
    extracted_features.append(features)

# Convert the list of features to a NumPy array
amuse_extracted_features = np.array(extracted_features)

# Print the extracted features
print("Feature Names:")
print(feature_names)
print("\nExtracted Features Shape:", amuse_extracted_features.shape)
print("Extracted Features for amuse array:")
print(amuse_extracted_features)



In [ ]:
# %%
import numpy as np

# Create three example NumPy arrays
array1 = baseline_extracted_features
array2 = stress_extracted_features
array3 = amuse_extracted_features
# Combine the arrays into one
features_array = np.concatenate((array1, array2, array3))

print(features_array)
print(len(features_array))


In [ ]:
# %%
pip install pandas


In [ ]:
# %%
import numpy as np
import pandas as pd

# Number of segments for each label
label_1_count = len(baseline_extracted_features)
label_2_count = len(stress_extracted_features)
label_3_count = len(amuse_extracted_features)


# Creating arrays for each label
labels_1 = np.ones(label_1_count)
labels_2 = np.full(label_2_count, 2)
labels_3 = np.full(label_3_count, 3)

# Combine all labels into one array
all_labels = np.concatenate([labels_1, labels_2, labels_3])
print((len(all_labels)))

# Optionally shuffle the labels if needed


# Create a DataFrame



In [ ]:
# %%
pip install scikit-learn


In [ ]:
# %%
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Load your feature data and labels
# Replace with your actual features and labels
features = features_array
labels = all_labels

# Print the lengths of features and labels
print('Number of samples in features:', len(features))
print('Number of samples in labels:', len(labels))

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Train and evaluate SVM model
svm_model = SVC(kernel='poly')
svm_model.fit(X_train, y_train)
y_val_pred_svm = svm_model.predict(X_val)
print('SVM Validation Accuracy:', accuracy_score(y_val, y_val_pred_svm))
print('SVM Validation Report:')
print(classification_report(y_val, y_val_pred_svm))

# Train and evaluate k-NN model
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
y_val_pred_knn = knn_model.predict(X_val)
print('k-NN Validation Accuracy:', accuracy_score(y_val, y_val_pred_knn))
print('k-NN Validation Report:')
print(classification_report(y_val, y_val_pred_knn))

# Train and evaluate Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)
y_val_pred_rf = rf_model.predict(X_val)
print('Random Forest Validation Accuracy:', accuracy_score(y_val, y_val_pred_rf))
print('Random Forest Validation Report:')
print(classification_report(y_val, y_val_pred_rf))

# Train and evaluate Logistic Regression model
log_reg_model = LogisticRegression(max_iter=500)
log_reg_model.fit(X_train, y_train)
y_val_pred_log_reg = log_reg_model.predict(X_val)
print('Logistic Regression Validation Accuracy:', accuracy_score(y_val, y_val_pred_log_reg))
print('Logistic Regression Validation Report:')
print(classification_report(y_val, y_val_pred_log_reg))

# Train and evaluate MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=500)
mlp_model.fit(X_train, y_train)
y_val_pred_mlp = mlp_model.predict(X_val)
print('MLP Validation Accuracy:', accuracy_score(y_val, y_val_pred_mlp))
print('MLP Validation Report:')
print(classification_report(y_val, y_val_pred_mlp))

# Evaluate the models on the test set
# Choose the best model based on validation performance
y_test_pred_rf =  mlp_model.predict(X_test)
print('MLP Test Accuracy:', accuracy_score(y_test, y_test_pred_rf))
print('MLP Forest Test Report:')
print(classification_report(y_test, y_test_pred_rf))



